## Importing Libraries

In [1]:
import pandas as pd
import mysql.connector
from credentials import db_password

## Connecting to **gdb023** database

In [2]:
mydb = mysql.connector.connect(host = 'localhost', user = 'root', password = db_password, database = 'gdb023')
cursorObject = mydb.cursor()

query = '''
        SHOW TABLES
        '''
cursorObject.execute(query)
tables_list = cursorObject.fetchall()

for table in tables_list:
    print(table[0])

dim_customer
dim_product
fact_gross_price
fact_manufacturing_cost
fact_pre_invoice_deductions
fact_sales_monthly


## Requests :

### Q1. Provide the list of markets in which customer **"Atliq Exclusive"** operates its business in the **APAC** region.

In [3]:
QUERY = '''
        SELECT DISTINCT(market) FROM dim_customer
            WHERE region = 'APAC' AND customer = 'Atliq Exclusive'
        '''
cursorObject.execute(QUERY)
pd.DataFrame(cursorObject.fetchall(), columns = cursorObject.column_names)

,market
0,India
1,Indonesia
2,Japan
3,Philiphines
4,South Korea
5,Australia
6,Newzealand
7,Bangladesh
